# Llama Stack Client Examples
Use the llama_stack_client library to interact with a Llama Stack server

First let's install the required packages

In [1]:
%pip install -U llama-stack-client
%pip install -U llama-stack
%pip install -U agentops
%pip install -U python-dotenv
%pip install -U fastapi
%pip install opentelemetry-api
%pip install opentelemetry-sdk


  Attempting uninstall: llama-stack-client
    Found existing installation: llama_stack_client 0.0.57
    Uninstalling llama_stack_client-0.0.57:
      Successfully uninstalled llama_stack_client-0.0.57
Note: you may need to restart the kernel to use updated packages.
  Using cached blobfile-3.0.0-py3-none-any.whl.metadata (15 kB)
  Using cached fire-0.7.0.tar.gz (87 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached pillow-11.0.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (9.1 kB)
  Using cached pycryptodomex-3.21.0-cp36-abi3-macosx_10_9_universal2.whl.metadata (3.4 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached MarkupSafe-3.0.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (4.0 kB)
  Using cached regex-2024.11.6-cp310-cp3

In [3]:
from llama_stack_client import LlamaStackClient
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.inference.event_logger import EventLogger
from llama_stack_client.types import UserMessage
from llama_stack_client.types.agent_create_params import AgentConfig
from llama_stack_client.lib.agents.agent import Agent
from dotenv import load_dotenv
import os
import agentops

load_dotenv()
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"

agentops.init(AGENTOPS_API_KEY, default_tags=["llama-stack-client-example"], auto_start_session=False)

host = "0.0.0.0" # LLAMA_STACK_HOST
port = 5001 # LLAMA_STACK_PORT

full_host = f"http://{host}:{port}"

client = LlamaStackClient(
    base_url=f"{full_host}",
)

🖇 AgentOps: API Key is invalid: {<your_agentops_key>}.
	    Find your API key at https://app.agentops.ai/settings/projects
🖇 AgentOps: API Key is invalid: {<your_agentops_key>}.
	    Find your API key at https://app.agentops.ai/settings/projects


🖇 AgentOps:  WARNING: agentops is out of date. Please update with the command: 'pip install --upgrade agentops'


# Inference Canary 1 - Completion with Streaming

In [4]:
agentops.start_session()
response = client.inference.chat_completion(
    messages=[
        UserMessage(
            content="hello world, write me a 3 word poem about the moon",
            role="user",
        ),
    ],
    model_id="meta-llama/Llama-3.2-1B-Instruct",
    stream=True
)

async for log in EventLogger().log(response):
    log.print()

agentops.end_session("Success")

🖇 AgentOps: API Key is invalid: {<your_agentops_key>}.
	    Find your API key at https://app.agentops.ai/settings/projects
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=ceea2686-c0ed-4190-b106-eeae88ffe5ca


Assistant> Silent lunar glow


🖇 AgentOps: API Key is invalid: {<your_agentops_key>}.
	    Find your API key at https://app.agentops.ai/settings/projects
🖇 AgentOps: Session Stats - Duration: 7.9s | Cost: $0.00 | LLMs: 1 | Tools: 0 | Actions: 0 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=ceea2686-c0ed-4190-b106-eeae88ffe5ca


# Inference Canary Example 2 - Completion without Streaming

In [5]:
agentops.start_session()
response = client.inference.chat_completion(
    messages=[
        UserMessage(
            content="write me a 3 word poem about the moon",
            role="user",
        ),
    ],
    model_id="meta-llama/Llama-3.2-1B-Instruct",
    stream=False
)

print(f"> Response: {response.completion_message.content}")
agentops.end_session("Success")

🖇 AgentOps: API Key is invalid: {<your_agentops_key>}.
	    Find your API key at https://app.agentops.ai/settings/projects
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=990b495b-e4c6-4c78-97d9-21dd47101ff3


InternalServerError: Error code: 500 - {'detail': 'Internal server error: An unexpected error occurred.'}

# Agent Canary Example

In [9]:
import os
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.types.agent_create_params import AgentConfig

agentops.start_session()

LLAMA_STACK_PORT = 5001

# Replace with actual API keys for functionality
BRAVE_SEARCH_API_KEY = os.getenv("BRAVE_SEARCH_API_KEY") or "your-brave-search-api-key"

async def agent_test():
    client = LlamaStackClient(
        base_url=f"http://0.0.0.0:{LLAMA_STACK_PORT}",
    )

    available_shields = [shield.identifier for shield in client.shields.list()]
    if not available_shields:
        print("No available shields. Disable safety.")
    else:
        print(f"Available shields found: {available_shields}")
    available_models = [model.identifier for model in client.models.list()]
    if not available_models:
        raise ValueError("No available models")
    else:
        selected_model = available_models[0]
        print(f"Using model: {selected_model}")

    agent_config = AgentConfig(
        model=selected_model,
        instructions="You are a helpful assistant. Just say hello as a greeting.",
        sampling_params={
            "strategy": "greedy",
            "temperature": 1.0,
            "top_p": 0.9,
        },
        tools=[
            {
                "type": "brave_search",
                "engine": "brave",
                "api_key": BRAVE_SEARCH_API_KEY,
            }
        ],
        tool_choice="auto",
        tool_prompt_format="json",
        input_shields=available_shields if available_shields else [],
        output_shields=available_shields if available_shields else [],
        enable_session_persistence=False,
    )
    agent = Agent(client, agent_config)
    user_prompts = [
        "Hello",
        "Which players played in the winning team of the NBA western conference semifinals of 2014, please use tools",
    ]

    session_id = agent.create_session("test-session")

    for prompt in user_prompts:
        response = agent.create_turn(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            session_id=session_id,
        )

        print(f"{response=}")

        for log in EventLogger().log(response):
            log.print()

agentops.start_session()

await agent_test()

agentops.end_session("Success")

🖇 AgentOps: API Key is invalid: {<your_agentops_key>}.
	    Find your API key at https://app.agentops.ai/settings/projects
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=48206eed-d5d8-4979-ab6e-3577faff5ad4
🖇 AgentOps: API Key is invalid: {<your_agentops_key>}.
	    Find your API key at https://app.agentops.ai/settings/projects
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=f0f95a35-876f-478d-9542-fe3261ad3d18


No available shields. Disable safety.
Using model: meta-llama/Llama-3.2-1B-Instruct
response=<generator object LlamaStackClientProvider.handle_response.<locals>.agent_generator at 0x10f44b370>
inference> Hello! How can I assist you today?

🖇 AgentOps: Multiple sessions detected. You must use session.record(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().



response=<generator object LlamaStackClientProvider.handle_response.<locals>.agent_generator at 0x10f44a3b0>
inference> brave_search.call(query="NBA Western Conference Semifinals 2014 teams")

🖇 AgentOps: Multiple sessions detected. You must use session.record(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().
🖇 AgentOps: Multiple sessions detected. You must use session.record(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().



tool_execution> Tool:brave_search Args:{'query': 'NBA Western Conference Semifinals 2014 teams'}
tool_execution> Tool:brave_search Response:{"query": "NBA Western Conference Semifinals 2014 teams", "top_k": [{"title": "2014 NBA playoffs - Wikipedia", "url": "https://en.wikipedia.org/wiki/2014_NBA_playoffs", "description": "This would be the last Game 7 won by a road <strong>team</strong> until the 2016 <strong>NBA</strong> Finals. Game 4 of the Heat\u2013Nets series saw LeBron James record a Heat franchise playoff high 49 points. He eventually led the Miami Heat to their fourth consecutive Eastern <strong>Conference</strong> Finals appearance with a win in Game 5. With a Game 5 win over the Portland Trail Blazers, the San Antonio Spurs advanced to the <strong>Western</strong> ...", "type": "search_result"}, {"title": "2014 NBA Western Conference Semifinals - Trail Blazers vs. Spurs | Basketball-Reference.com", "url": "https://www.basketball-reference.com/playoffs/2014-nba-western-conf

🖇 AgentOps: Multiple sessions detected. You must use session.record(). More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management
🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().


🖇 AgentOps: API Key is invalid: {<your_agentops_key>}.
	    Find your API key at https://app.agentops.ai/settings/projects
🖇 AgentOps: Could not end session - multiple sessions detected. You must use session.end_session() instead of agentops.end_session() More info: https://docs.agentops.ai/v1/concepts/core-concepts#session-management


In [8]:
agentops.end_all_sessions()

🖇 AgentOps: Session Stats - Duration: 28.8s | Cost: $0.00 | LLMs: 0 | Tools: 0 | Actions: 0 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=990b495b-e4c6-4c78-97d9-21dd47101ff3
🖇 AgentOps: Session Stats - Duration: 19.6s | Cost: $0.00 | LLMs: 0 | Tools: 0 | Actions: 0 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=516a6f7f-56b5-4f04-bad6-a42d76fc7f55
🖇 AgentOps: Session Stats - Duration: 9.8s | Cost: $0.00 | LLMs: 0 | Tools: 0 | Actions: 0 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=e6a248fb-b78c-4fd4-bffe-50a0a8065bfa
